In [1]:
from operator import methodcaller

import pandas as pd

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Load data

In [2]:
datasets = {
    dataset_name: catalog.load(dataset_name) for dataset_name in catalog.layers["raw"]
}
set(datasets)

2020-11-17 10:47:52,428 - kedro.io.data_catalog - INFO - Loading data from `raw_third_health_camp_attended` (CSVDataSet)...
2020-11-17 10:47:52,437 - kedro.io.data_catalog - INFO - Loading data from `raw_patient_profile` (CSVDataSet)...
2020-11-17 10:47:52,562 - kedro.io.data_catalog - INFO - Loading data from `raw_first_health_camp_attended` (CSVDataSet)...
2020-11-17 10:47:52,572 - kedro.io.data_catalog - INFO - Loading data from `raw_train` (CSVDataSet)...
2020-11-17 10:47:52,719 - kedro.io.data_catalog - INFO - Loading data from `raw_health_camp_detail` (CSVDataSet)...
2020-11-17 10:47:52,730 - kedro.io.data_catalog - INFO - Loading data from `raw_test` (CSVDataSet)...
2020-11-17 10:47:52,784 - kedro.io.data_catalog - INFO - Loading data from `raw_second_health_camp_attended` (CSVDataSet)...


{'raw_first_health_camp_attended',
 'raw_health_camp_detail',
 'raw_patient_profile',
 'raw_second_health_camp_attended',
 'raw_test',
 'raw_third_health_camp_attended',
 'raw_train'}

# Explore data

In [3]:
datasets["raw_train"]

,,Registration_Date,Var1,Var2,Var3,Var4,Var5
Patient_ID,Health_Camp_ID,,,,,,
489652,6578,2005-09-10,4,0,0,0,2
507246,6578,2005-08-18,45,5,0,0,7
523729,6534,2006-04-29,0,0,0,0,0
524931,6535,2004-02-07,0,0,0,0,0
521364,6529,2006-02-28,15,1,0,0,7
...,...,...,...,...,...,...,...
500969,6539,2005-01-03,0,0,0,0,0
511952,6528,2006-02-13,0,0,0,0,0
521236,6554,2005-05-24,0,0,0,0,0


In [4]:
datasets["raw_train"].describe(include="all")

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-4-711a9de6b3bb>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  datasets["raw_train"].describe(include="all")


,Registration_Date,Var1,Var2,Var3,Var4,Var5
count,74944,75278.000000,75278.000000,75278.000000,75278.000000,75278.000000
unique,1201,NaN,NaN,NaN,NaN,NaN
top,2006-03-28 00:00:00,NaN,NaN,NaN,NaN,NaN
freq,868,NaN,NaN,NaN,NaN,NaN
first,2003-10-19 00:00:00,NaN,NaN,NaN,NaN,NaN
last,2007-02-09 00:00:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.796156,0.238556,0.000385,0.022424,0.248386
std,NaN,7.626687,3.769594,0.019624,0.199570,1.213938
min,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000


"Camps started on or before 31st March 2006 are considered in Train," so why is `Registration_date` frequently after 31st March 2006? 

In [5]:
(datasets["raw_train"].Registration_Date > pd.to_datetime("3/31/2006")).value_counts(dropna=False)

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


False    66382
True      8896
Name: Registration_Date, dtype: int64

In [6]:
datasets["raw_train"].iloc[:, 1:].apply(
    methodcaller("value_counts", dropna=False)
) / len(datasets["raw_train"])

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Var1,Var2,Var3,Var4,Var5
0,0.895946,0.967507,0.999615,0.984803,0.920867
1,0.034565,0.013111,0.000385,0.009525,0.028707
2,0.018372,0.007041,NaN,0.004291,0.015091
3,0.014373,0.003773,NaN,0.001209,0.012288
4,0.007506,0.001355,NaN,0.000173,0.007426
...,...,...,...,...,...
259,0.000013,NaN,NaN,NaN,NaN
271,0.000027,NaN,NaN,NaN,NaN
277,0.000013,NaN,NaN,NaN,NaN
286,0.000027,NaN,NaN,NaN,NaN


In [7]:
datasets["raw_test"]

,,Registration_Date,Var1,Var2,Var3,Var4,Var5
Patient_ID,Health_Camp_ID,,,,,,
505701,6548,2006-05-21,1,0,0,0,2
500633,6584,2006-06-02,0,0,0,0,0
506945,6582,2006-08-10,0,0,0,0,0
497447,6551,2006-08-27,0,0,0,0,0
496446,6533,2006-09-19,0,0,0,0,0
...,...,...,...,...,...,...,...
507261,6525,2006-04-13,12,2,0,0,6
513607,6551,2006-11-03,0,0,0,0,0
498160,6584,2006-06-17,0,0,0,0,0


In [8]:
datasets["raw_test"].describe(include="all")

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-7-067155d123fb>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  datasets["raw_test"].describe(include="all")


,Registration_Date,Var1,Var2,Var3,Var4,Var5
count,35249,35249.000000,35249.000000,35249.000000,35249.000000,35249.000000
unique,375,NaN,NaN,NaN,NaN,NaN
top,2006-09-18 00:00:00,NaN,NaN,NaN,NaN,NaN
freq,632,NaN,NaN,NaN,NaN,NaN
first,2006-01-31 00:00:00,NaN,NaN,NaN,NaN,NaN
last,2007-02-09 00:00:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.819201,0.253312,0.000567,0.041845,0.258163
std,NaN,8.298817,3.984690,0.023814,0.268826,1.311230
min,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
(datasets["raw_train"].Registration_Date <= pd.to_datetime("3/31/2006")).value_counts(dropna=False)

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


True     66048
False     9230
Name: Registration_Date, dtype: int64

In [10]:
datasets["raw_test"].iloc[:, 1:].apply(
    methodcaller("value_counts", dropna=False)
) / len(datasets["raw_test"])

,Var1,Var2,Var3,Var4,Var5
0,0.897671,0.964878,0.999433,0.969304,0.920452
1,0.033249,0.013816,0.000567,0.023206,0.027433
2,0.017589,0.008823,NaN,0.004709,0.015348
3,0.014242,0.004170,NaN,0.002099,0.013249
4,0.008312,0.001645,NaN,0.000596,0.006497
...,...,...,...,...,...
297,0.000028,NaN,NaN,NaN,NaN
299,0.000028,NaN,NaN,NaN,NaN
319,0.000028,NaN,NaN,NaN,NaN
339,0.000057,NaN,NaN,NaN,NaN


## Health_Camp_Detail.csv

In [11]:
datasets["raw_health_camp_detail"]

,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
Health_Camp_ID,,,,,
6560,2003-08-16,2003-08-20,First,B,2
6530,2003-08-16,2003-10-28,First,C,2
6544,2003-11-03,2003-11-15,First,F,1
6585,2003-11-22,2003-12-05,First,E,2
6561,2003-11-30,2003-12-18,First,E,1
...,...,...,...,...,...
6556,2006-11-09,2006-12-15,First,E,2
6551,2006-11-13,2006-11-18,Second,D,2
6559,2006-12-11,2007-03-12,First,F,2


In [12]:
datasets["raw_health_camp_detail"].index.get_level_values("Health_Camp_ID").nunique()

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


65

In [13]:
datasets["raw_health_camp_detail"].Category1.value_counts(dropna=False)

First     44
Second    11
Third     10
Name: Category1, dtype: int64

In [14]:
datasets["raw_health_camp_detail"].Category2.value_counts(dropna=False)

C    16
E    15
F    11
G    10
A     7
D     4
B     2
Name: Category2, dtype: int64

`Category2` is a subcategory of `Category1`.

In [15]:
datasets["raw_health_camp_detail"].groupby("Category1").Category2.value_counts(dropna=False)

Category1  Category2
First      C            16
           E            15
           F            11
           B             2
Second     A             7
           D             4
Third      G            10
Name: Category2, dtype: int64

`Category3` is highly imbalanced.

In [16]:
datasets["raw_health_camp_detail"].Category3.value_counts(dropna=False)

2    62
1     3
Name: Category3, dtype: int64

### Assess linkability

In [17]:
datasets["raw_train"].merge(
    datasets["raw_health_camp_detail"], how="outer", on="Health_Camp_ID", indicator=True
)._merge.value_counts()

both          75278
right_only       21
left_only         0
Name: _merge, dtype: int64

In [18]:
datasets["raw_test"].merge(
    datasets["raw_health_camp_detail"], how="outer", on="Health_Camp_ID", indicator=True
)._merge.value_counts()

both          35249
right_only       44
left_only         0
Name: _merge, dtype: int64

## Patient_Profile.csv

In [19]:
datasets["raw_patient_profile"]

,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
Patient_ID,,,,,,,,,,
516956,0,0,0,0,1.0,90.0,39.0,2003-06-18,NaN,Software Industry
507733,0,0,0,0,1.0,NaN,40.0,2003-07-20,H,Software Industry
508307,0,0,0,0,3.0,87.0,46.0,2002-11-02,D,BFSI
512612,0,0,0,0,1.0,75.0,47.0,2002-11-02,D,Education
521075,0,0,0,0,3.0,NaN,80.0,2002-11-24,H,Others
...,...,...,...,...,...,...,...,...,...,...
518600,0,0,0,0,NaN,NaN,NaN,2007-01-28,NaN,NaN
509515,0,0,0,0,NaN,NaN,NaN,2007-01-27,NaN,NaN
510713,0,0,0,0,NaN,NaN,NaN,2007-02-01,H,NaN


Online engagement is highly imbalanced/sparse.

In [20]:
datasets["raw_patient_profile"].Online_Follower.value_counts(dropna=False)

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    36785
1      848
Name: Online_Follower, dtype: int64

In [21]:
datasets["raw_patient_profile"].LinkedIn_Shared.value_counts(dropna=False)

0    36614
1     1019
Name: LinkedIn_Shared, dtype: int64

In [22]:
datasets["raw_patient_profile"].Twitter_Shared.value_counts(dropna=False)

0    36820
1      813
Name: Twitter_Shared, dtype: int64

In [23]:
datasets["raw_patient_profile"].Facebook_Shared.value_counts(dropna=False)

0    36747
1      886
Name: Facebook_Shared, dtype: int64

`Income` is over 88% missing.

In [24]:
datasets["raw_patient_profile"].Income.value_counts(dropna=False)

NaN    33271
0.0     2033
1.0     1013
2.0      685
3.0      362
4.0      156
5.0       70
6.0       43
Name: Income, dtype: int64

`Education_Score` is over 96% missing.

In [25]:
datasets["raw_patient_profile"].Education_Score.value_counts(dropna=False)

NaN          36249
75.000000       57
82.000000       57
79.000000       52
76.000000       51
             ...  
91.566667        1
67.933333        1
77.726667        1
84.500000        1
74.966667        1
Name: Education_Score, Length: 228, dtype: int64

`Age` is 87% missing.

In [26]:
datasets["raw_patient_profile"].Age.value_counts(dropna=False)

NaN     32618
41.0      361
43.0      359
42.0      336
40.0      321
39.0      299
44.0      293
45.0      240
38.0      219
72.0      218
46.0      214
73.0      199
47.0      189
37.0      186
71.0      175
74.0      172
48.0      140
49.0      127
36.0      109
50.0      102
51.0       76
70.0       71
52.0       66
35.0       66
53.0       53
75.0       43
34.0       42
54.0       40
69.0       36
55.0       35
76.0       32
56.0       25
58.0       21
77.0       18
33.0       17
57.0       16
67.0       15
68.0       14
59.0       13
60.0       10
32.0        7
61.0        7
63.0        6
65.0        6
78.0        5
66.0        4
64.0        4
62.0        3
31.0        2
30.0        2
80.0        1
Name: Age, dtype: int64

In [27]:
datasets["raw_patient_profile"].First_Interaction.value_counts(dropna=False)

2004-09-08    574
2005-05-01    544
2005-05-21    498
2005-02-09    437
2004-10-25    305
             ... 
2004-06-10      1
2005-11-20      1
2004-02-13      1
2003-12-14      1
2004-04-02      1
Name: First_Interaction, Length: 1476, dtype: int64

`City_Type` is over 62% missing.

In [28]:
datasets["raw_patient_profile"].City_Type.value_counts(dropna=False)

NaN    23384
B       2662
H       2517
A       1729
D       1704
G       1352
E       1217
C       1216
I       1036
F        816
Name: City_Type, dtype: int64

### Assess linkability

In [29]:
datasets["raw_train"].merge(
    datasets["raw_patient_profile"], how="outer", on="Patient_ID", indicator=True
)._merge.value_counts()

both          75278
right_only     7805
left_only         0
Name: _merge, dtype: int64

In [30]:
datasets["raw_test"].merge(
    datasets["raw_patient_profile"], how="outer", on="Patient_ID", indicator=True
)._merge.value_counts()

both          35249
right_only    22309
left_only         0
Name: _merge, dtype: int64

## First_Health_Camp_Attended.csv

In [31]:
datasets["raw_first_health_camp_attended"]

,,Donation,Health_Score,Unnamed: 4
Patient_ID,Health_Camp_ID,,,
506181,6560,40,0.439024,NaN
494977,6560,20,0.097561,NaN
518680,6560,10,0.048780,NaN
509916,6560,30,0.634146,NaN
488006,6560,20,0.024390,NaN
...,...,...,...,...
502728,6575,10,0.509804,NaN
511088,6575,20,0.078431,NaN
507608,6575,40,0.627451,NaN


`Donation` is never missing or zero (or under 10).

In [32]:
datasets["raw_first_health_camp_attended"].Donation.value_counts(dropna=False)

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


20     1609
10     1316
30     1250
40      760
50      473
60      283
70      194
80       91
90       74
100      51
110      30
120      24
130      18
140      13
150       8
160       8
170       6
250       2
180       2
210       2
280       1
220       1
190       1
330       1
Name: Donation, dtype: int64

In [33]:
datasets["raw_first_health_camp_attended"].Health_Score.value_counts(dropna=False)

0.360339    111
0.464856    102
0.559855    101
0.753325     91
0.752137     58
           ... 
0.489627      1
0.897764      1
0.919890      1
0.132780      1
0.919087      1
Name: Health_Score, Length: 3826, dtype: int64

In [34]:
datasets["raw_first_health_camp_attended"].Health_Score.dtype

dtype('float64')

In [35]:
datasets["raw_first_health_camp_attended"].Health_Score.isna().sum()

0

### Assess linkability

In [36]:
datasets["raw_patient_profile"].merge(
    datasets["raw_first_health_camp_attended"],
    how="outer",
    on="Patient_ID",
    indicator=True,
)._merge.value_counts()

left_only     34085
both           6218
right_only        0
Name: _merge, dtype: int64

In [37]:
datasets["raw_health_camp_detail"].merge(
    datasets["raw_first_health_camp_attended"],
    how="outer",
    on="Health_Camp_ID",
    indicator=True,
)._merge.value_counts()

both          6218
left_only       31
right_only       0
Name: _merge, dtype: int64

## Second_Health_Camp_Attended.csv

In [38]:
datasets["raw_second_health_camp_attended"]

,,Health Score
Patient_ID,Health_Camp_ID,
526631,6536,0.875136
509122,6536,0.755700
498864,6536,0.673181
515398,6536,0.722041
504624,6536,0.464712
...,...,...
490853,6529,0.103136
499836,6529,0.507841
511286,6529,0.627865


In [39]:
datasets["raw_second_health_camp_attended"]["Health Score"].value_counts(dropna=False)

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.402054    1018
0.373206     545
0.455371     135
0.505134     126
0.065742     109
            ... 
0.987697       1
0.998815       1
0.998633       1
0.998420       1
0.997949       1
Name: Health Score, Length: 208, dtype: int64

In [40]:
datasets["raw_second_health_camp_attended"]["Health Score"].dtype

dtype('float64')

In [41]:
datasets["raw_second_health_camp_attended"]["Health Score"].isna().sum()

0

### Assess linkability

In [42]:
datasets["raw_patient_profile"].merge(
    datasets["raw_second_health_camp_attended"],
    how="outer",
    on="Patient_ID",
    indicator=True,
)._merge.value_counts()

left_only     31510
both           7819
right_only        0
Name: _merge, dtype: int64

In [43]:
datasets["raw_health_camp_detail"].merge(
    datasets["raw_second_health_camp_attended"],
    how="outer",
    on="Health_Camp_ID",
    indicator=True,
)._merge.value_counts()

both          7819
left_only       59
right_only       0
Name: _merge, dtype: int64

## Third_Health_Camp_Attended.csv

In [44]:
datasets["raw_third_health_camp_attended"]

Number_of_stall_visited  Last_Stall_Visited_Number
Patient_ID Health_Camp_ID                                                    
517875     6527                                  3                          1
504692     6578                                  1                          1
           6527                                  3                          1
493167     6527                                  4                          4
510954     6528                                  2                          2
...                                            ...                        ...
503791     6527                                  3                          3
517319     6528                                  1                          3
           6578                                  4                          3
           6527                                  4                          4
511234     6527                                  5                          4

[6515 rows x 2 columns]

In [45]:
datasets["raw_third_health_camp_attended"].Number_of_stall_visited.value_counts(dropna=False)

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1    1822
2    1243
3    1145
5    1034
4     701
6     536
0      18
7      16
Name: Number_of_stall_visited, dtype: int64

In [46]:
datasets["raw_third_health_camp_attended"].Last_Stall_Visited_Number.value_counts(dropna=False)

1    2497
2    1290
3    1234
4     753
5     472
6     245
0      18
7       6
Name: Last_Stall_Visited_Number, dtype: int64

In [47]:
datasets["raw_third_health_camp_attended"][
    datasets["raw_third_health_camp_attended"].Number_of_stall_visited == 0
]

,,Number_of_stall_visited,Last_Stall_Visited_Number
Patient_ID,Health_Camp_ID,,
486530,6527,0,0
492766,6527,0,0
510061,6527,0,0
514624,6527,0,0
506873,6527,0,0
489505,6527,0,0
503497,6527,0,0
514108,6527,0,0
515663,6527,0,0


### Assess linkability

In [48]:
datasets["raw_patient_profile"].merge(
    datasets["raw_third_health_camp_attended"],
    how="outer",
    on="Patient_ID",
    indicator=True,
)._merge.value_counts()

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


left_only     32293
both           6515
right_only        0
Name: _merge, dtype: int64

In [49]:
datasets["raw_health_camp_detail"].merge(
    datasets["raw_third_health_camp_attended"],
    how="outer",
    on="Health_Camp_ID",
    indicator=True,
)._merge.value_counts()

both          6515
left_only       61
right_only       0
Name: _merge, dtype: int64

# Construct target variable

In [50]:
target1 = (
    datasets["raw_train"]
    .join(datasets["raw_first_health_camp_attended"])
    .Health_Score.notna()
)
target1

Patient_ID  Health_Camp_ID
489652      6578              False
507246      6578              False
523729      6534              False
524931      6535              False
521364      6529              False
                              ...  
500969      6539              False
511952      6528              False
521236      6554               True
518817      6580              False
511284      6555              False
Name: Health_Score, Length: 75278, dtype: bool

In [51]:
target1.value_counts()

False    69060
True      6218
Name: Health_Score, dtype: int64

In [52]:
target2 = (
    datasets["raw_train"]
    .join(datasets["raw_second_health_camp_attended"])["Health Score"]
    .notna()
)
target2

Patient_ID  Health_Camp_ID
489652      6578              False
507246      6578              False
523729      6534               True
524931      6535              False
521364      6529               True
                              ...  
500969      6539              False
511952      6528              False
521236      6554              False
518817      6580              False
511284      6555              False
Name: Health Score, Length: 75278, dtype: bool

In [53]:
target2.value_counts()

False    67459
True      7819
Name: Health Score, dtype: int64

In [54]:
target3 = (
    datasets["raw_train"]
    .join(datasets["raw_third_health_camp_attended"])
    .Number_of_stall_visited > 0
)
target3

Patient_ID  Health_Camp_ID
489652      6578               True
507246      6578              False
523729      6534              False
524931      6535              False
521364      6529              False
                              ...  
500969      6539              False
511952      6528               True
521236      6554              False
518817      6580              False
511284      6555              False
Name: Number_of_stall_visited, Length: 75278, dtype: bool

In [55]:
target3.value_counts()

/anaconda3/envs/kedro-session/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


False    68781
True      6497
Name: Number_of_stall_visited, dtype: int64

In [56]:
pd.DataFrame({"target1": target1, "target2": target2, "target3": target3}).sum(
    axis=1
).value_counts()

0    54744
1    20534
dtype: int64